In [17]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import os
import time
import tensorflow as tf

from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras import backend as K
from keras.preprocessing import image                  
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0

In [18]:
def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [19]:
diseases = [
    'Cardiomegaly','Emphysema','Effusion',
    'Hernia','Nodule','Pneumothorax',
    'Atelectasis','Pleural_Thickening',
    'Mass','Edema','Consolidation',
    'Infiltration','Fibrosis','Pneumonia',
    'No Finding'
]

dataset_df = pd.read_csv('./dataset_information/Data_Entry_2017.csv')

In [20]:
# Applying One Hot Encoding to Labels
for disease in diseases:
    dataset_df[disease] = dataset_df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

In [21]:
image_labels = dataset_df[diseases].to_numpy()
image_paths = {
    os.path.basename(x): x for x in glob(os.path.join('.', 'images', '*.png'))
}

print(f"Samples Found: {len(image_paths)}")

Samples Found: 112120


In [5]:
# Storing path to each image name in the dataframe
dataset_df['Image Path'] = dataset_df['Image Index'].map(image_paths.get)

In [6]:
images_list = dataset_df['Image Path'].tolist()

labelB = (dataset_df[diseases].sum(axis = 1) > 0).tolist()
labelB = np.array(labelB, dtype = int)

In [7]:
def read_image_to_tensor(path, shape):
    # Loads RGB image to PIL format
    img = image.load_img(path, target_size = shape)
    
    # Convert PIL image to 3D tensor of specific shape
    # and normalizes it by dividing each pixel by 255
    normalized_image_tensor = image.img_to_array(img) / 255
    
    # Convert 3D tensor to 4D tensor with specific shape 
    # (1, shape, 3) and return it
    return np.expand_dims(normalized_image_tensor, axis = 0)

In [8]:
def image_to_array(paths, shape):
    images_arrays = []
    for path in tqdm(paths, desc = "Progress", ncols = 100):
        images_arrays.append(read_image_to_tensor(path, shape))
    return images_arrays

In [9]:
train_labels = labelB[ : 75000][ : , np.newaxis]
valid_labels = labelB[75000 : 85000][ : , np.newaxis]
test_labels = labelB[85000 : ][ : , np.newaxis]

In [50]:
IMAGE_SHAPE = (64, 64)

training_samples = image_to_array(images_list[ : 75000], shape = IMAGE_SHAPE)
validation_samples = image_to_array(images_list[75000 : 85000], shape = IMAGE_SHAPE)
test_samples = image_to_array(images_list[85000 : ], shape = IMAGE_SHAPE)

Progress:   1%|▍                                                | 714/75000 [00:09<16:53, 73.31it/s]


KeyboardInterrupt: 

In [ ]:
# Creating a model with EfficientNet B0 as base.
efficient_net_b0 = EfficientNetB0(
    weights = 'imagenet',
    include_top = False,
    input_shape = training_samples[1 : ]
)

custom_classifier = Sequential()
custom_classifier.add(GlobalAveragePooling2D(input_shape = efficient_net_b0.output_shape[1 : ]))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(512, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(50, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs = efficient_net_b0.input, output = add_model(custom_classifier.output))
model.summary()

In [14]:
# Defining 2 optimizers to test the model with.

optimizer_1 = tf.keras.optimizers.SGD(
    learning_rate = 1e-4, 
    decay = 1e-6, 
    momentum = 0.9, 
    nesterov = True
)
optimizer_2 = tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
)

In [15]:
# Compiling model with loss function, optimizer and metrics

model.compile(
    optimizer = optimizer_1,
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        precision_threshold(threshold = 0.5), 
        recall_threshold(threshold = 0.5), 
        fbeta_score_threshold(beta=0.5, threshold = 0.5)
    ]
)

NameError: name 'model' is not defined

In [ ]:
# Defining some hyper-parameters

EPOCHS = 5
BATCH_SIZE = 32

In [ ]:
# Defining object for augmentation 

train_datagen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False 
)

In [ ]:
%%timeit -n1 -r1

history = model.fit_generator(
    train_datagen.flow(
        training_samples, 
        train_labels, 
        batch_size = BATCH_SIZE
    ),
    steps_per_epoch = len(training_samples) // BATCH_SIZE,
    validation_data = (validation_samples, valid_labels),
    validation_steps = len(validation_samples) // BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [checkpointer, log], 
    verbose = 1
)

In [ ]:
prediction = model.predict(test_samples)

In [ ]:
threshold = 0.5
beta = 0.5

accuracy = K.eval(binary_accuracy(K.variable(value=test_labels), K.variable(value=prediction)))
precision = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
recall = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
f1_score = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))

In [ ]:
print (f"Accuracy: {accuracy} \nPrecision: {precision} \nRecall: {recall} \nF1-Score: {f1_score}")